In [14]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(user='ndr_spa',
                        password='ndr-spa',
                        host='172.29.253.105',
                        port="31602",
                        database='spa-paas')
df = pd.DataFrame()

with conn.cursor() as cur:

# Просмотрело пользователей
    cur.execute("""
 SELECT COALESCE (t_spa_merging.merged_spa_id,t_dashboard_data_spa.spa_id) AS "User",
       t_dashboard_data_spa."timestamp" AS "Date",
       t_dashboard_data_spa.custom_event_id AS "Action"
        ,t_channel.channelname as t_channel_channelname,
       t_ua_browser.family as t_ua_browser_family,
       t_ua_os.family as t_ua_os_family
FROM metrics.dashboard_data_spa as t_dashboard_data_spa
         LEFT JOIN metrics.ua_os as t_ua_os ON t_dashboard_data_spa.os_id = t_ua_os.id
         LEFT JOIN metrics.ua_browser as t_ua_browser ON t_dashboard_data_spa.browser_id = t_ua_browser.id
         LEFT JOIN metrics.channel as t_channel ON t_dashboard_data_spa.channel_name_typeid = t_channel.channel_name_typeid
         LEFT JOIN metrics.spa_merging t_spa_merging ON t_dashboard_data_spa.spa_id = t_spa_merging.spa_id
WHERE
        t_dashboard_data_spa.date_timestamp >= 1672617600.000
  AND t_dashboard_data_spa.date_timestamp <= 1675295999.999
  AND t_dashboard_data_spa.counterid = 172
ORDER BY 1, 2, 3
""")

    data = cur.fetchall()
    columns = list(cur.description)
    datacolumns=[c[0] for c in columns]
    df = pd.DataFrame(data=data, columns = datacolumns)
    
#df.head(100)  
df

,User,Date,Action,t_channel_channelname,t_ua_browser_family,t_ua_os_family
0,5610410,1675253818,816458943053036507,SPA WebSDK ver. 1.0,Chrome,Windows
1,5610410,1675253831,1078481309569142348,SPA WebSDK ver. 1.0,Chrome,Windows
2,5610410,1675253832,5968215246386644958,SPA WebSDK ver. 1.0,Chrome,Windows
3,5610410,1675253834,2663893037261661396,SPA WebSDK ver. 1.0,Chrome,Windows
4,5610410,1675253835,5968215246386644958,SPA WebSDK ver. 1.0,Chrome,Windows
...,...,...,...,...,...,...
104,5610410,1675253915,8963603339518700811,SPA WebSDK ver. 1.0,Chrome,Windows
105,5610410,1675253916,-3078137023039660317,SPA WebSDK ver. 1.0,Chrome,Windows
106,5610410,1675253916,2288299052224170527,SPA WebSDK ver. 1.0,Chrome,Windows
107,5610410,1675253918,-7515928846097923806,SPA WebSDK ver. 1.0,Chrome,Windows


In [15]:
# 1 ШАГ - ВЫГРУЗКА ДАННЫХ

#выгружаем дата сет из csv
# import pandas as pd
# df = pd.read_csv('C:/Users/Maria.Ruchkina/Desktop/Исходные данные2.csv', encoding='windows-1251', sep=';')

# df['Date'] = pd.to_datetime(df['Date'])
# df['Action'] = df['Action'].astype(str)

#сортируем данные сначала по пользователю, зачем, для каждого пользователя - по дате
df=df.sort_values(by=['User', 'Date', 'Action'])

df.reset_index(inplace = True, drop = True)

#добавляем столбцы - Случай
df["Case"] = ''
#добавляем столбцы - Шаг
df["Step"] = ''
#добавляем столбец - Первый раз
df["FirstTime"] = ''

df.to_csv("исходные данные.csv", sep=';', encoding='windows-1251')
df

,User,Date,Action,t_channel_channelname,t_ua_browser_family,t_ua_os_family,Case,Step,FirstTime
0,5610410,1675253818,816458943053036507,SPA WebSDK ver. 1.0,Chrome,Windows,,,
1,5610410,1675253831,1078481309569142348,SPA WebSDK ver. 1.0,Chrome,Windows,,,
2,5610410,1675253832,5968215246386644958,SPA WebSDK ver. 1.0,Chrome,Windows,,,
3,5610410,1675253834,2663893037261661396,SPA WebSDK ver. 1.0,Chrome,Windows,,,
4,5610410,1675253835,5968215246386644958,SPA WebSDK ver. 1.0,Chrome,Windows,,,
...,...,...,...,...,...,...,...,...,...
104,5610410,1675253915,8963603339518700811,SPA WebSDK ver. 1.0,Chrome,Windows,,,
105,5610410,1675253916,-3078137023039660317,SPA WebSDK ver. 1.0,Chrome,Windows,,,
106,5610410,1675253916,2288299052224170527,SPA WebSDK ver. 1.0,Chrome,Windows,,,
107,5610410,1675253918,-7515928846097923806,SPA WebSDK ver. 1.0,Chrome,Windows,,,


In [16]:
#создаем список шагов

steps =[816458943053036507, -5017003053612302618, -8520731022752367714]


In [17]:
# 2 ШАГ - ДЛЯ ПРОСТОГО ПОРЯДКА - РАЗМЕЧАЕМ СОБЫТИЯ В ДАТА СЕТЕ

# caseNumber - счетчик для случаев
caseNumber = 1

# счетчик для шагов
stepNumber = 0

# цикл по шагам воронки
for step in steps:
    stepNumber = stepNumber + 1
    
    # цикл по списку событий
    for i, row in df.iterrows():
        
        # работаем с записью только если для нее не определен шаг
        if row['Step'] == '':
            
            # если у записи событие = текущему шагу воронки
            if row['Action'] == step:
                
                # Пропускаем текущее событие, если:
                # Это не первая запись ИЛИ у предыдущей записи тот же юзер, событие и для него определен тот же шаг или шаг не опрделен
                # Это необходимо, так как если записи о событии от одного и того же пользователя идут подряд, мы считаем это одним и тем же шагом, а также для для того, чтобы не пропустить шаг, если для 2 шагов указано одно и то же событие
                
                if i==0 or (df.iloc[i-1]['User'] == row['User'] and df.iloc[i-1]['Action'] == step and (df.iloc[i-1]['Step'] == stepNumber or df.iloc[i-1]['Step'] == '')) == False:
                    
                    # если текущий шаг - первый, фиксируем случай и шаг
                    if stepNumber == 1:
                        df.at[i, 'Case']=caseNumber
                        df.at[i, 'Step'] = stepNumber
                        df.at[i, 'FirstTime']=True
                        caseNumber = caseNumber + 1
                    # если текущий шаг - не первый
                    else:
                        # ищем предыдущий шаг, так как шаги должны идти друг за другом, а так же чтобы идентифицировать случай
                        
                        prev_steps = df[(df['User'] == row['User']) & (df['Step'] == (stepNumber-1)) & (df['Date'] < row['Date'])]
                        
                        # если нашли записи, которые могут быть предыдущими шагами
                        
                        if prev_steps.empty == False: 
                            # получаем самую последнюю по дате запись - это и будет предыдущий шаг для текущей воронки
                         #   idxmax = prev_steps[['Date']].idxmax()
                         #   prev_step=prev_steps.loc[idxmax].iloc[0]
                            
                            prev_step=prev_steps.tail(n=1).iloc[0]
 
                            # если нашли предыдущий шаг
                            if prev_step.empty == False:
                                # ищем, не был ли уже определен текущий шаг для соотв. случая
                                sameStep = df[(df['User'] == row['User']) & (df['Step'] == stepNumber) & (df['Case'] == prev_step['Case'])]
                                # если не нашли такой же шаг
                                if sameStep.empty == True:
                                    breakingStep = df[(df['User'] == row['User']) & (df['Step'] == 1) & (df['Date'] < row['Date']) & (df['Date'] > prev_step['Date'])]
                                    if breakingStep.empty == True:
                                        df.at[i, 'Case'] = prev_step['Case']
                                        df.at[i, 'Step'] = stepNumber
                                        df.at[i, 'FirstTime']=True
                                        
df_res=df[df.FirstTime == True]
df.to_csv("размеченные данные.csv", sep=';', encoding='windows-1251')

df_res

,User,Date,Action,t_channel_channelname,t_ua_browser_family,t_ua_os_family,Case,Step,FirstTime
0,5610410,1675253818,816458943053036507,SPA WebSDK ver. 1.0,Chrome,Windows,1,1,True
33,5610410,1675253873,-5017003053612302618,SPA WebSDK ver. 1.0,Chrome,Windows,1,2,True
57,5610410,1675253880,-8520731022752367714,SPA WebSDK ver. 1.0,Chrome,Windows,1,3,True


In [18]:
# ФОРМИРУЕМ СЛУЧАИ

cases_list = pd.DataFrame({'Case': df_res.Case.unique()})
cases_list["User"]=''

step_number = 0

for step in steps:
    step_number = step_number + 1
    if step_number != 1:
        cases_list["Diff_Step"+str(step_number)] = ''
    cases_list["Step"+str(step_number)] = ''
    
for i, case in cases_list.iterrows():
    step_number = 0
    for step in steps:
        step_number = step_number + 1
        step_row = df_res[(df_res['Case'] == case['Case']) & (df_res['Step'] == step_number)]
        if step_row.empty == False:
            cases_list.at[i, "Step" + str(step_number)]=step_row.iloc[0].Date
            cases_list.at[i, 'User']=step_row.iloc[0].User
            if step_number != 1:
                cases_list.at[i, "Diff_Step" + str(step_number)] = (cases_list.at[i, "Step" + str(step_number)] - cases_list.at[i, "Step" + str(step_number-1)])
                
cases_list

,Case,User,Step1,Diff_Step2,Step2,Diff_Step3,Step3
0,1,5610410,1675253818,55,1675253873,7,1675253880


In [19]:
# ОТОБРАЖАЕМ МЕТРИКИ

step_number = 0

for step in steps:
    step_number = step_number + 1    
    print("Шаг "+ str(step_number)+":")
    step_df=cases_list[cases_list["Step"+str(step_number)] != '']
    print("Количество случаев ="+ str(step_df.shape[0]))
    print("Количество пользователей = " + str(step_df.User.unique().shape[0]))
    if step_number != 1:
        print("Среднее время после предыдущего шага = " + str(step_df["Diff_Step" + str(step_number)].mean()))
        print("Среднее время после предыдущего шага (медиана) = " + str(step_df["Diff_Step" + str(step_number)].median()))

Шаг 1:
Количество случаев =1
Количество пользователей = 1
Шаг 2:
Количество случаев =1
Количество пользователей = 1
Среднее время после предыдущего шага = 55.0
Среднее время после предыдущего шага (медиана) = 55.0
Шаг 3:
Количество случаев =1
Количество пользователей = 1
Среднее время после предыдущего шага = 7.0
Среднее время после предыдущего шага (медиана) = 7.0
